# Different types of Tracking system
- Python dictionaries, CSV files, print outs
- TensorBoard       https://www.tensorflow.org/tensorboard/get_started
- Weights && Biases https://wandb.ai/site/experiment-tracking
- MLFlow  https://mlflow.org/



In [1]:
# 1. Setup
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be .13+"
    
    print(f"torch vision: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")  

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.0.1
torchvision version: 0.15.2


In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("Didn't find torchinfo, installing...")
    !pip install -q torchinfo
    from torchinfo import summary
    
    
try:
    from going_modular.going_modular import data_setup, engine
except:
    print("Didn't find going_modular scripts... downloading from github")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Set seeds
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [5]:
# get data
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool=True) -> Path:
    
    # Setup path to data folder
    data_path = Path("data/")
    img_path = data_path / destination
    
    # if the img folder doesn't exist, download it and prepare it...
    if img_path.is_dir():
        print("Dir exists, skipping...")
        
    else:
        print(f"Didn't find {img_path}, creating one...")
        img_path.mkdir(parents=True, exist_ok=True)
        
        # Download data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"Download {target_file} from {source}")
            f.write(request.content)
        
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"Unzipping {target_file}")
            zip_ref.extractall(img_path)
            
        if remove_source:
            os.remove(data_path / target_file)
            
    return img_path    

In [6]:
img_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                         destination="pizza_steak_sushi",remove_source=True)
img_path

Dir exists, skipping...


PosixPath('data/pizza_steak_sushi')

In [7]:
img_path

PosixPath('data/pizza_steak_sushi')

In [13]:
# Datasets and dataloaders            -> automatically
train_dir = img_path / "train"
test_dir = img_path / "test"

# Setup pretrained weights 
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# transformer from weights
automatic_transforms = weights.transforms()
print(f"Automatically created transforms -> {automatic_transforms}")

# dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=automatic_transforms,
    batch_size=32
)

train_dataloader, test_dataloader, class_names

Automatically created transforms -> ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x7ff763454490>,
 ['pizza', 'steak', 'sushi'])

In [15]:
# Datasets and dataloaders            -> manualy
train_dir = img_path/"train"
test_dir = img_path/"test"

# mean and std -> ImageNet parameters
normalize = transforms.Normalize(mean=[.485, .456, .406],
                                 std=[.29, .224, .225])

manual_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

print(f"Manually created transforms -> {manual_transforms}")

# dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = manual_transforms,
    batch_size = 32
)

train_dataloader, test_dataloader, class_names


Manually created transforms -> Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.29, 0.224, 0.225])
)


(<torch.utils.data.dataloader.DataLoader at 0x7ff762604750>,
 ['pizza', 'steak', 'sushi'])

In [16]:
# Pretrained model

# model = torchvision.models.efficientnet_b0(pretrained=True).to(device)  # old


weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT              # DEFAULT -> best 
model = torchvision.models.efficientnet_b0(weights=weights).to(device)    # new


model

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /home/loveplay1983/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 20.5M/20.5M [00:07<00:00, 2.95MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [27]:
# Freeze all base layers by setting requires_grad to False

for param in model.features.parameters():
    param.requires_grad = False
    
set_seeds()

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names),
              bias=True)
).to(device)

In [28]:
from torchinfo import summary

summary(model, 
        input_size=(32, 3, 224, 224),
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=15,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape     Output Shape    Param #         Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224] [32, 3]         --              Partial
├─Sequential (features)                                      [32, 3, 224, 224] [32, 1280, 7, 7] --              False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224] [32, 32, 112, 112] --              False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224] [32, 32, 112, 112] (864)           False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112] [32, 32, 112, 112] (64)            False
│    │    └─SiLU (2)                                         [32, 32, 112, 112] [32, 32, 112, 112] --              --
│    └─Sequential (1)                                        [32, 32, 112, 112] [32, 16, 112, 112] --              False
│    │    └─MBConv (0)                      

In [29]:
# Train model and track results

In [30]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

In [33]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [ ]:
from typing import Dict, List
from dqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      
    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for 
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]} 
      For example if training for epochs=2: 
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]} 
    """
    # Create empty result dictionary
    results = {"train_loss: " [],
               "train_acc: ": [],
               "test_loss: ": [],
               "test_acc: ": []
              }
    
    # loop through training and testing steps for a num of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        
        print(
        
            f"Epoch -> {epoch+1} |"
            f"train_loss -> {train_loss:.4f} | "
            f"train_acc -> {train_acc: .4f} | "
            f"test_loss -> {test_loss: .4f} |"
            f"test_acc -> {test_acc: .4f}"
        )
        
        # update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        ############ experiment tracking  ################################
        # Add loss results to SummaryWriter
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch
        )
        
        # Add acc results to SummaryWriter
        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc},
                           global_step=epoch
        )
        
        # Add the tracking of PyTorch model architecture
        writer.add_graph(model=model,
                         )
        